# Notebook 1 - Fondamentaux Python pour l'IA
# Analyse de données météorologiques en temps réel

🎯 Objectifs pédagogiques

Maîtriser les structures de données Python essentielles
Consommer des APIs REST avec requests
Manipuler des données JSON et CSV
Créer des visualisations basiques
Appliquer des statistiques descriptives

🌤️ Contexte du projet

Vous travaillez pour une startup AgTech qui développe des solutions d'agriculture intelligente. Votre mission : analyser les données météorologiques de plusieurs villes européennes pour optimiser les recommandations de plantation.

Partie 1 : Connexion aux APIs météo

🔧 Installation des bibliothèques


 À exécuter dans votre terminal ou cellule
# pip install requests pandas matplotlib seaborn numpy

In [36]:
#📥 Import et configuration
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
from dotenv import load_dotenv
import os

load_dotenv()

True

🌍 API OpenWeatherMap (gratuite)

Inscription : Créez un compte sur openweathermap.org


Clé API : Récupérez votre clé gratuite (40 000 appels/mois)



In [73]:
### 💡 Première requête guidée
api_key_owm = os.getenv('API_KEY_OWM')

def get_city_coordinates(city: str) -> dict:
    '''
    Récupère les coordonnées (latitude et longitude) d'une ville donnée en utilisant l'API OpenWeatherMap.

    args :
        city (str) : nom de la ville à rechercher.

    returns :
        dict : dictionnaire contenant les informations de la ville dont la latitude ('lat') et la longitude ('lon').
        None : si la ville n'est pas trouvée ou en cas d'erreur.
    '''
    try:
        url = 'http://api.openweathermap.org/geo/1.0/direct'
        params = {
            'q': city,
            'appid': api_key_owm,
            'limit': 1
        }

        response = requests.get(url, params=params)

        # gérer l'erreur 401
        if response.status_code == 401:
            print('401 client error: Invalid API key')
            return None

        data_raw: dict = response.json()[0]
        return data_raw
    
    # gérer le cas où une ville n'est pas valide
    except IndexError:
        print(f'\'{city}\' is not a valid city')
    except Exception as e:
        print(f'error: {e}')

def get_weather_data(city):
    """
    Récupère les données météo actuelles pour une ville

    Étapes à compléter :
    1. Construire l'URL avec les paramètres
    2. Faire l'appel API avec requests.get()
    3. Vérifier le status code
    4. Retourner les données JSON
    """
    try:
        city_data = get_city_coordinates(city)

        if city_data == None:
            print(f'Error: Couldn\'t get city data for \'{city}\'')
            return None

        latitude = city_data['lat']
        longitude = city_data['lon']

        url = 'https://api.openweathermap.org/data/2.5/weather'
        params = {
            'lat': latitude,
            'lon': longitude,
            'appid': api_key_owm,
            'units': 'metric',
            'lang': 'fr'
        }

        response = requests.get(url, params=params)

        print(f'status code: {response.status_code}')

        return response.json()

    except Exception as e:
        print(f'error: {e}')

weather_in_montpellier = get_weather_data('montpellier')
print(weather_in_montpellier)

status code: 200
{'coord': {'lon': 3.8809, 'lat': 43.6107}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'ciel dégagé', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 24.16, 'feels_like': 23.78, 'temp_min': 24.16, 'temp_max': 25.71, 'pressure': 1014, 'humidity': 44, 'sea_level': 1014, 'grnd_level': 1005}, 'visibility': 10000, 'wind': {'speed': 8.23, 'deg': 340}, 'clouds': {'all': 0}, 'dt': 1753697495, 'sys': {'type': 2, 'id': 2038454, 'country': 'FR', 'sunrise': 1753676962, 'sunset': 1753729941}, 'timezone': 7200, 'id': 2992166, 'name': 'Montpellier', 'cod': 200}


In [75]:
# Villes à analyser (agriculture européenne)
CITIES = ["Paris", "Berlin", "Madrid", "Rome", "Amsterdam", "Vienna"]

for city in CITIES:
    weather = get_weather_data(city)
    print(f'weather in {city}: {weather}')

status code: 200
weather in Paris: {'coord': {'lon': 2.32, 'lat': 48.8589}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'couvert', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 19.92, 'feels_like': 19.8, 'temp_min': 19.26, 'temp_max': 20.58, 'pressure': 1020, 'humidity': 70, 'sea_level': 1020, 'grnd_level': 1011}, 'visibility': 10000, 'wind': {'speed': 3.6, 'deg': 330}, 'clouds': {'all': 100}, 'dt': 1753696806, 'sys': {'type': 1, 'id': 6550, 'country': 'FR', 'sunrise': 1753676324, 'sunset': 1753731328}, 'timezone': 7200, 'id': 6545270, 'name': 'Quartier du Palais-Royal', 'cod': 200}
status code: 200
weather in Berlin: {'coord': {'lon': 13.3889, 'lat': 52.517}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'nuageux', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 19.99, 'feels_like': 19.92, 'temp_min': 19.99, 'temp_max': 21.01, 'pressure': 1009, 'humidity': 72, 'sea_level': 1009, 'grnd_level': 1005}, 'visibility': 10000, 'wind': {'speed': 6.69, 'de

**Questions de débogage :**
- Que faire si l'API retourne une erreur 401 ?
- Comment gérer une ville introuvable ?

---

## Partie 2 : API complémentaire - Données historiques

### 📊 API Visual Crossing Weather (gratuite)
Alternative avec 1000 appels/jour gratuits : [visualcrossing.com](https://www.visualcrossing.com/weather-api)

In [39]:
# Configuration Visual Crossing
VC_API_KEY = "VOTRE_CLE_VISUAL_CROSSING"
VC_BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"

def get_historical_weather(city, start_date, end_date, api_key):
    """
    Récupère les données météo historiques

    Défis à résoudre :
    1. Construire l'URL avec les dates
    2. Gérer la pagination si nécessaire
    3. Extraire les données pertinentes du JSON complexe
    4. Convertir en DataFrame pandas
    """

    url = f"{VC_BASE_URL}/{city}/{start_date}/{end_date}"

    params = {
        'key': api_key,
        'include': 'days',
        'elements': 'datetime,temp,humidity,precip,windspeed,pressure'
    }

    # Votre implémentation ici
    # Pensez à la gestion d'erreurs !

    pass

### 🎯 Mission pratique
Récupérez les données des 7 (et non 30) derniers jours pour toutes vos villes et créez un DataFrame consolidé.

**Structure attendue :**
```
| date       | ville     | temperature | humidite | precipitation | vent |
|------------|-----------|-------------|----------|---------------|------|
| 2024-01-01 | Paris     | 12.5        | 75       | 2.3          | 15   |
```

---


In [40]:
## Partie 3 : API supplémentaire - Qualité de l'air

### 🌬️ API OpenWeatherMap Air Pollution
def get_air_quality(lat, lon, api_key):
    """
    Récupère les données de qualité de l'air

    URL : http://api.openweathermap.org/data/2.5/air_pollution

    Étapes :
    1. Utiliser les coordonnées lat/lon des villes
    2. Récupérer l'indice AQI et les composants (PM2.5, PM10, O3, etc.)
    3. Joindre ces données avec vos données météo
    """
    pass

In [41]:
### 🗺️ Obtenir les coordonnées
def get_city_coordinates(city, api_key):
    """
    Utilise l'API Geocoding d'OpenWeatherMap
    URL : http://api.openweathermap.org/geo/1.0/direct

    Récupérez lat/lon pour chaque ville
    """
    pass


## Partie 4 : Analyse et visualisation

### 📈 Analyses à réaliser

1. **Comparaison inter-villes**
   - Températures moyennes par ville
   - Variabilité climatique (écart-type)
   - Corrélations température/humidité

2. **Tendances temporelles**
   - Évolution sur 30 jours
   - Identification des patterns

3. **Qualité de l'air vs météo**
   - Impact de la pluie sur la pollution
   - Corrélations vent/qualité de l'air

In [42]:
### 💡 Visualisations guidées

# 1. Heatmap des températures par ville et jour
plt.figure(figsize=(15, 8))

# Créez un pivot table : villes en colonnes, dates en lignes
# Utilisez seaborn.heatmap()

# 2. Boxplot comparatif des précipitations
# Utilisez seaborn.boxplot()

# 3. Scatter plot qualité air vs température
# Ajoutez une regression line avec seaborn.regplot()



<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

## Partie 5 : API bonus - Données agricoles

### 🌱 API AgroMonitoring (gratuite)

In [43]:
# API satellite pour l'agriculture
AGRO_API_KEY = "VOTRE_CLE_AGROMONITORING"

def get_soil_data(polygon_coordinates, api_key):
    """
    Récupère des données de sol via satellite
    URL : http://api.agromonitoring.com/agro/1.0/

    Données disponibles :
    - Indices de végétation (NDVI)
    - Humidité du sol
    - Température de surface
    """
    pass



**Défi avancé :** Créez des recommandations de plantation basées sur :
- Données météo des 30 derniers jours
- Prévisions à 5 jours
- Qualité de l'air
- Indices de végétation satellite



## 🏆 Livrables attendus

### 📊 Dashboard météo
Créez un tableau de bord contenant :
1. **Aperçu temps réel** des 6 villes
2. **Graphiques de tendances** sur 30 jours
3. **Alertes qualité de l'air** (AQI > 100)
4. **Recommandations agricoles** par ville

In [44]:
### 📱 Format de présentation

def generate_weather_report(city_data):
    """
    Génère un rapport automatisé

    Format :
    - Résumé exécutif (3 lignes)
    - Métriques clés (tableaux)
    - Graphiques (4 visualisations)
    - Recommandations (bullet points)
    """
    pass

---

## 🎓 Critères d'évaluation

- [ ] **APIs fonctionnelles** : Toutes les connexions API marchent
- [ ] **Gestion d'erreurs** : Code robuste avec try/except
- [ ] **Qualité des données** : Validation et nettoyage
- [ ] **Visualisations** : Graphiques informatifs et esthétiques
- [ ] **Insights business** : Recommandations basées sur les données

### 🔗 Préparation au Notebook 2
Le prochain notebook utilisera une vraie base de données PostgreSQL hébergée pour analyser des données de ventes e-commerce, en croisant avec vos données météo pour des analyses géolocalisées.

### 📚 APIs alternatives (si quotas dépassés)
- **WeatherAPI** : 1M appels/mois gratuits
- **AccuWeather** : 50 appels/jour gratuits  
- **Climatiq** : Données climat et carbone
- **NASA APIs** : Données satellite gratuites